In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.datasets import load_iris
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import VarianceThreshold


In [8]:
df = pd.read_csv("C:\\Users\Ali\\DataScience_Projects\\Kaggle_Competition\\train_set.csv")
df.head()

,RecordId,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X70,X71,X72,X73,X74,X75,X76,X77,X78,Y
0,1,87.000000,34.118411,0,2,0,165.100000,1,829,2,...,0.040000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2,82.372284,31.573280,0,0,1,162.983897,1,724,0,...,0.033431,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,3,50.000000,27.771653,0,0,1,165.100000,1,895,2,...,0.010000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,4,66.236109,26.515922,0,0,1,167.009549,1,637,0,...,0.039363,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,5,81.303299,20.843691,0,0,1,158.165419,0,564,0,...,0.069242,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [67]:
target_column = df['Y']

# Step 2: Apply variance filter
threshold = 0.7  # Set your variance threshold
# Calculate variance for each feature
variances = df.drop(columns='Y').var()
# Select features with variance above the threshold
features_to_keep = variances[variances > threshold].index
# Filter the DataFrame
df_filtered = df[features_to_keep.tolist() + ['Y']]

# Optionally, you can reattach the target column if you need to:
df_filtered['Y'] = target_column[df_filtered.index]

C:\Users\Ali\AppData\Local\Temp\ipykernel_20928\4270158449.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Y'] = target_column[df_filtered.index]


In [9]:
imr = SimpleImputer(missing_values=np.nan, strategy='mean')
imr = imr.fit(df.values)
imputed_data = imr.transform(df.values)
df= pd.DataFrame(imputed_data)

In [10]:
#Normalize
# Sample data
data = df

# Initialize the scaler
scaler = MinMaxScaler()

# Fit and transform the data
df = scaler.fit_transform(data)


In [28]:
df.head()

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [12]:
# Separating the target variable
X = df[:, 1:78] # 1 - 78 is 77 features... (78 is exclusive)
Y = df[:, 78]

# Splitting the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.15, random_state=42)


In [13]:
model = GradientBoostingClassifier(n_estimators=150, learning_rate=0.1, max_depth=3, random_state=42)
model.fit(X_train, y_train)

KeyboardInterrupt: 

In [93]:
y_pred = model.predict(X_test)

In [94]:
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

In [95]:
accuracy

0.9966142094856307

In [96]:
md_predictions_probs = model.predict_proba(X_test)
md_predictions_probs 

array([[9.98210125e-01, 1.78987461e-03],
       [9.98135519e-01, 1.86448104e-03],
       [9.99349890e-01, 6.50110287e-04],
       ...,
       [9.97488664e-01, 2.51133575e-03],
       [9.99349890e-01, 6.50110287e-04],
       [9.99007610e-01, 9.92390101e-04]])

In [14]:
fpr, tpr, _ = roc_curve(y_test, md_predictions_probs, pos_label=1)
plt.plot(fpr, tpr, marker = '.')

ValueError: Found input variables with inconsistent numbers of samples: [49225, 98450]

In [97]:
test_data = pd.read_csv("C:\\Users\Ali\\DataScience_Projects\\Kaggle_Competition\\test_set.csv")

In [98]:
recordId = test_data['RecordId']
test_data = test_data.drop(columns = 'RecordId')

In [83]:
#selector = VarianceThreshold(0.7)
#test_data= selector.fit_transform(test_data)

In [99]:
imr = SimpleImputer(missing_values=np.nan, strategy='mean')
imr = imr.fit(test_data)
imputed_data = imr.transform(test_data)
test_data = pd.DataFrame(imputed_data)

In [100]:
# Sample data
data = test_data

# Initialize the scaler
scaler = MinMaxScaler()

# Fit and transform the data
test_data = scaler.fit_transform(data)

In [101]:
prediction = model.predict_proba(test_data)
prediction= prediction[:,1]

In [102]:
finaldf = pd.concat([recordId, pd.DataFrame(prediction, columns=['Y'])], axis=1)
finaldf
finaldf.to_csv("C:\\Users\Ali\\DataScience_Projects\\Kaggle_Competition\\test_results.csv", index= False)